In [ ]:

!pip install gensim
!pip install nltk
!pip install numpy==1.25.2
!pip install datasets


import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from gensim.downloader import load as gensim_load
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import ttest_ind
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
# ---- Load MRPC Dataset ----
print("Loading MRPC dataset...")
dataset = load_dataset("glue", "mrpc")
mrpc = dataset["train"]
sentences1 = mrpc["sentence1"]
sentences2 = mrpc["sentence2"]
labels = mrpc["label"]

# ---- TF-IDF + Cosine Similarity ----
def compute_tfidf_similarity(sentences1, sentences2):
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    all_sentences = sentences1 + sentences2
    vectorizer.fit(all_sentences)
    s1 = vectorizer.transform(sentences1)
    s2 = vectorizer.transform(sentences2)
    return [cosine_similarity(a, b)[0][0] for a, b in zip(s1, s2)]

# ---- Word2Vec + Cosine Similarity ----
def compute_word2vec_similarity(sentences1, sentences2):
    w2v_model = gensim_load('word2vec-google-news-300')
    def sentence_vector(sentence):
        tokens = word_tokenize(sentence.lower())
        vectors = [w2v_model[w] for w in tokens if w in w2v_model]
        return np.mean(vectors, axis=0) if vectors else np.zeros(300)
    s1 = [sentence_vector(sent) for sent in sentences1]
    s2 = [sentence_vector(sent) for sent in sentences2]
    return [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0][0] for a, b in zip(s1, s2)]

# ---- BERT + Cosine Similarity ----
def compute_bert_similarity(sentences1, sentences2):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    s1 = model.encode(sentences1, convert_to_numpy=True)
    s2 = model.encode(sentences2, convert_to_numpy=True)
    return [cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0][0] for a, b in zip(s1, s2)]

# ---- Similarity to Binary Predictions ----
def to_predictions(similarities, threshold=0.75):
    return [1 if s >= threshold else 0 for s in similarities]

# ---- Classification Evaluation ----
def evaluate_classification(name, preds, labels):
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    print(f"{name} ➤ Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

# ---- Welch’s t-test ----
def run_welchs_test(scores_a, scores_b, label_a="Model A", label_b="Model B"):
    t_stat, p_val = ttest_ind(scores_a, scores_b, equal_var=False)
    print(f"{label_a} vs {label_b} ➤ t = {t_stat:.4f}, p = {p_val:.4f}")
    if p_val < 0.05:
        print("→ Statistically significant difference ✅\n")
    else:
        print("→ No statistically significant difference ❌\n")

# ---- Run All ----
print("Computing similarities...")

# Compute Similarities
tfidf_sim = compute_tfidf_similarity(sentences1, sentences2)
word2vec_sim = compute_word2vec_similarity(sentences1, sentences2)
bert_sim = compute_bert_similarity(sentences1, sentences2)

# Binary predictions
tfidf_preds = to_predictions(tfidf_sim)
word2vec_preds = to_predictions(word2vec_sim)
bert_preds = to_predictions(bert_sim)

# Evaluate
print("\nModel Evaluation on MRPC (1000 pairs):")
evaluate_classification("TF-IDF", tfidf_preds, labels)
evaluate_classification("Word2Vec", word2vec_preds, labels)
evaluate_classification("BERT", bert_preds, labels)

# Welch’s t-test
print("\nWelch’s t-test between models (similarity scores):")
run_welchs_test(tfidf_sim, word2vec_sim, "TF-IDF", "Word2Vec")
run_welchs_test(word2vec_sim, bert_sim, "Word2Vec", "BERT")
run_welchs_test(tfidf_sim, bert_sim, "TF-IDF", "BERT")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Loading MRPC dataset...
Computing similarities...

Model Evaluation on MRPC (1000 pairs):
TF-IDF ➤ Accuracy: 0.4299 | F1 Score: 0.2856
Word2Vec ➤ Accuracy: 0.6903 | F1 Score: 0.8110
BERT ➤ Accuracy: 0.7246 | F1 Score: 0.7935

Welch’s t-test between models (similarity scores):
TF-IDF vs Word2Vec ➤ t = -111.1915, p = 0.0000
→ Statistically significant difference ✅

Word2Vec vs BERT ➤ t = 38.9491, p = 0.0000
→ Statistically significant difference ✅

TF-IDF vs BERT ➤ t = -65.3459, p = 0.0000
→ Statistically significant difference ✅



In [ ]:
# ---- Compute Mean and Standard Deviation ----
def compute_stats(similarities, name):
    mean_val = np.mean(similarities)
    sd_val = np.std(similarities)
    median_val = np.median(similarities)
    print(f"{name} ➤ Mean: {mean_val:.4f} | SD: {sd_val:.4f} | Median: {median_val:.4f}")
    return mean_val, sd_val, median_val

# ---- Print Mean, SD, Median for All Models ----
print("\nSimilarity Score Statistics:")
tfidf_mean, tfidf_sd, tfidf_median = compute_stats(tfidf_sim, "TF-IDF")
word2vec_mean, word2vec_sd, word2vec_median = compute_stats(word2vec_sim, "Word2Vec")
bert_mean, bert_sd, bert_median = compute_stats(bert_sim, "BERT")



Similarity Score Statistics:
TF-IDF ➤ Mean: 0.5258 | SD: 0.1872 | Median: 0.5297
Word2Vec ➤ Mean: 0.8909 | SD: 0.0670 | Median: 0.9035
BERT ➤ Mean: 0.7850 | SD: 0.1504 | Median: 0.8168
